In [1]:
%%writefile exploration.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE stackoverflow_;

DESCRIBE posts;

SELECT year, tag
FROM posts
LATERAL VIEW EXPLODE(tags) tags AS tag
LIMIT 10;

Overwriting exploration.hql


In [2]:
! hive -f exploration.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 1.06 seconds
OK
id                  	int                 	                    
post_type_id        	tinyint             	                    
date                	string              	                    
owner_user_id       	int                 	                    
parent_id           	int                 	                    
score               	int                 	                    
favorite_count      	int                 	                    
tags                	array<string>       	                    
year                	string 

In [3]:
%%writefile query.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE stackoverflow_;

WITH questions AS (
                   SELECT tag, year, COUNT(*) AS cnt
                   FROM posts
                   LATERAL VIEW EXPLODE(tags) tags AS tag
                   WHERE (post_type_id = 1) AND (year IN (2009, 2016))
                   GROUP BY tag, year
)
SELECT CONCAT_WS('\t', q16.tag, STRING(q16.rk), STRING(q09.rk), STRING(q16.cnt), STRING(q09.cnt))
FROM (
      SELECT tag, cnt, RANK() OVER (ORDER BY cnt DESC) AS rk
      FROM questions
      WHERE year = 2016
      ORDER BY cnt DESC
      LIMIT 10
) AS q16
LEFT OUTER JOIN (
                 SELECT tag, cnt, RANK() OVER (ORDER BY cnt DESC) AS rk
                 FROM questions
                 WHERE year = 2009  
) AS q09
ON q16.tag = q09.tag;

Overwriting query.hql


In [4]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.019 seconds
Query ID = jovyan_20180922011717_043e27c7-453c-4e1a-8c04-7c1512580d02
Total jobs = 7
Launching Job 1 out of 7
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1537574219280_0007, Tracking URL = http://03ed5f5ba941:8088/proxy/application_1537574219280_0007/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1537574219280_0007
Hadoop job information for